In [236]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.doc2vec import Doc2Vec
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import sys
import os
import numpy as np
import json
import random
import re

In [237]:
def GetTokens(text_content):
    
    ans = []
    # p = PorterStemmer()
    text = text_content.replace("-",":")
    text_content = text.replace("\n",":")
    tokenized_content = re.split(r'''[ `',.=:(_);{}?`"\n]''',text_content)
    for token in tokenized_content:
        token = token.lower()
        if (len(token)>2):
            if (token not in stop_words and not re.search('[0-9]+',token)):
                # token = p.stem(token,0,len(token)-1)
                ans.append(token)
                    
    return ans

In [247]:
def GetDocumentList(CollectionName):

    docContent = {}
    required_tags = ["text"]
    all_dir = tqdm(os.listdir(os.path.join("tipster_comp",CollectionName)),position=0, leave=True)
    for num in all_dir:
        filepath = os.path.join(os.path.join("tipster_comp",CollectionName),num)
        with open(filepath,"r",errors="ignore") as f:
            content = f.read()   
            bs_content = bs(content,'html.parser')
            all_doc = bs_content.find_all('doc')
            for doc in all_doc:
                docno = doc.find("docno").get_text().strip()
                for tags in required_tags:
                    text_content = doc.find_all(str(tags))
                    tokentext = ""
                    for text in text_content:
                        doc_content = text.get_text()
                        tokentext+=doc_content
                    docContent[docno] = tokentext

    return docContent

In [248]:
def GetStopwords(file):

    stopwordFile = open(os.path.join("tipster_comp",file))
    stop_words = set()
    stop_words.add('&')
    stop_words.add('=')
    stop_words.add('"')
    stop_words.add('<')
    stop_words.add('>')
    lines = stopwordFile.readlines()
    words = " ".join(lines)
    words_list = re.split('''[\n]''',words)

    for word in words_list:
        stop_words.add(word)

    return stop_words

In [249]:
def getModel(model):
    modelname = Doc2Vec.load(model)
    return modelname

In [250]:
def getDocVector(model,doc):
    vec = model.dv[doc]
    return np.array(vec)

In [251]:
def getVector(words,modelname):

    model = getModel(modelname)
    v1 = model.infer_vector(words,epochs=1)
    return np.array(v1)

In [252]:
def getModel(model):
    modelname = Doc2Vec.load(model)
    return modelname

In [278]:
def ExtractQrels(filename):

    qrels = []
    with open(filename,"r") as o:
        lines = o.readlines()
        text = " ".join(lines)
        text = text.replace("\n"," ")
        text = text.replace("\\"," ")
        text = text.replace("}"," ")
        text = text.replace("{"," ")
        text = re.split(" ",text)

    for word in text:
        if "AP" in word:
            qrels.append(word)
            # print(word)
    return qrels

In [279]:
def RandomSampleTest(data):
    
    qrels = ExtractQrels("qrels_AP.rtf")
    test_data = {}
    # print(test_data)
    for doc in qrels:
        if doc not in test_data:
            test_data[doc] = data[doc]
#             print(doc)
#     print("done")
    return test_data

In [348]:
def GetQueries(filename):

    queries = {}
    query_file = os.path.join(os.getcwd(),"queries","topics","trec12",filename)
    loop = tqdm(total=50)
    with open(query_file,"r",errors="ignore") as f:
        content = f.read()

        bs_content = bs(content,'html.parser')
        all_queries = bs_content.find_all('top')

        for query in all_queries:

            qno = query.find("num").get_text()
            qno = GetQueryNumber(qno)

            content = query.find("title")
            
            text = content.get_text()

            queries[qno] = text
            loop.update(1)

    return queries

In [349]:
def CosineSim(vector1,vector2):
    cosine_similarity = 1 - spatial.distance.cosine(np.array(vector1,dtype=np.float32), np.array(vector2,dtype=np.float32))

    return abs(cosine_similarity)

In [350]:
def GetQueryNumber(text):

    tokens = text.split("Number:")
    for word in tokens:
        word = word.strip()
        if (len(word)>0):
            return word

In [351]:
queries = GetQueries("topics.51-100.doc")

100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 2174.03it/s]


In [352]:
def getQueryDict(queries,model):
    vect = {}
    loop = tqdm(total=50)
    for query in queries:
        tokens = GetTokens(queries[query])
        vect[query] = getQueryVector(tokens,model)
        # print(vect[query].shape)
        loop.update(1)

    return vect

In [353]:
def GetStopwords(file):

    stopwordFile = open(os.path.join("tipster_comp",file))
    stop_words = set()
    stop_words.add('&')
    stop_words.add('=')
    stop_words.add('"')
    stop_words.add("topic")
    stop_words.add('<')
    stop_words.add('>')
    lines = stopwordFile.readlines()
    words = " ".join(lines)
    words_list = re.split('''[\n]''',words)

    for word in words_list:
        stop_words.add(word)

    return stop_words

In [354]:
stop_words = GetStopwords("stopwords.txt")

In [355]:
def getQueryVector(query,model):
    vec = getVector(query,model)
    return vec

In [356]:
def getVector(words,modelname):

#     model = getModel(modelname)
    v1 = model.infer_vector(words,epochs=1)
    return np.array(v1)

In [357]:
query_dict = getQueryDict(queries,model)

100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 2700.36it/s]


In [358]:
print(query_dict)

{'051': array([ 1.77091807e-02,  2.13729427e-03,  7.94296805e-03,  5.55808935e-03,
       -6.20888872e-03, -8.50828178e-03,  1.51958829e-02,  1.92192325e-03,
       -1.58105511e-03,  8.00865237e-03,  5.49662020e-03, -6.14303071e-03,
       -1.60178617e-02,  1.02063864e-02,  9.39434487e-03, -2.87229242e-03,
       -3.37383430e-03,  2.21498008e-03, -2.69825384e-03, -1.58449765e-02,
        1.38005046e-02, -5.33195771e-03,  3.42288823e-03,  5.17817587e-03,
        7.99241010e-03, -1.07987765e-02,  1.52821783e-02, -1.96653814e-03,
       -2.18826439e-02,  3.28555796e-03,  2.74980962e-02, -2.70804483e-03,
        1.24529563e-02, -6.91796094e-03,  3.26689752e-03, -7.01278821e-03,
        9.18253604e-03,  3.55893141e-03, -6.59231609e-03,  4.46337089e-03,
       -1.02169076e-02,  5.44124469e-03, -3.63632780e-03,  8.68358742e-03,
       -1.35842115e-02, -5.08703617e-03, -5.34489332e-03, -1.16571691e-03,
       -1.57193560e-03,  1.04254764e-02, -5.98181831e-03, -3.49237025e-03,
        4.9117016

model.most_similar(docID)

In [359]:
def loadScoresqrel(
    result_file
):  
    scores = {}
    with open(result_file) as f:
        lines = f.read().splitlines()
        for line in lines:
            words = re.split("\t", line)
            # print(words)
#             print(words)
            # print(line, result_file)
            query_id = words[0]
            doc_id = words[2]
            if not query_id in scores:
                scores[query_id] = []
            scores[query_id].append(doc_id)
    return scores

In [360]:
qid = '085'

In [361]:
query_vec = query_dict[qid]

In [362]:
sim_docs = model.dv.most_similar(positive= [query_vec])

In [468]:
def rank_list(dataset,cluster):
    
    query_map = {}
    
    if (dataset == "ap"):
        if (cluster=="kmeans"):
            filename = "K-Means_Doc2Vec_AP.txt"
        else:
            filename = "Clarans_Doc2Vec_AP.txt"
    else:
        if (cluster == "kmeans"):
            filename = "K-Means_Doc2Vec_ZF.txt"
        else:
            filename = "Clarans_Doc2Vec_ZF.txt"
            
    with open(os.path.join("Rankings",filename),"r") as o:
        lines = o.readlines()
        text = " ".join(lines)
        text = text.split("\n")

        for line in text:
            words = line.split()
            if (len(words)==3):
                qno = words[0]
                doc = words[1]
                score = words[2]
                if qno in query_map:
                    query_map[qno].append(doc)
                else:
                    query_map[qno] = [doc]
                
    return query_map

In [441]:
ranking_kmeans_bert_ap = kmeans_ranks()

In [489]:
def investigate(qno,docs,docids):
    total = len(docids)
    print("Chosen Query" +":"+ queries[qno])
    print("First Retrieved Document" +":"+ docs[docids[0]])
    print("50th Retrieved Document"+":"+docs[docids[int(total/2)]])
    print("100th Retrieved Document" +":"+ docs[docids[-1]])

In [490]:
ranklists = ["Clarans_Doc2Vec_AP.txt","Clarans_Doc2Vec_ZF.txt","K-Means_Doc2Vec_ZF.txt","K-Means_Doc2Vec_AP.txt"]
q1 = [("099","061","078"),("051","052","053")]
q2 = [("075","098","066"),("078","080","099")]
q3 = [("098","066","065"),("074","077","078")]
q4 = [("099","061","085"),("077","081","084")]
query_kind = {"clarans_ap":q1,"clarans_zf":q2,"kmeans_zf":q3,"kmeans_ap":q4}

In [463]:
docs_ap = GetDocumentList("ap")
modelname_ap = "d2vap.model"
model_ap = getModel(modelname)

100%|███████████████████████████████████████| 1051/1051 [01:16<00:00, 13.70it/s]


In [464]:
docs_zf = GetDocumentList("ziff")
modelname_zf = "d2vzf.model"
model_zf = getModel(modelname)

100%|█████████████████████████████████████████| 786/786 [01:44<00:00,  7.52it/s]


In [491]:
def show_queries(dataset,typedoc,cluster):
    
    docids =  rank_list(dataset,cluster)
    
    if (dataset == "ap"):
        docs = docs_ap
        if (cluster == "clarans"):
            print("Showing Output Rankings of CLARANS :-")
            queries = query_kind["clarans_ap"]
            good = queries[0]
            bad = queries[1]
            
            if (typedoc=="good"):
                print("Showing Good Queries:- ")
                for qno in good:
                    investigate(qno,docs,docids[qno])
            else:
                print("Showing Bad Queries:- ")
                for qno in bad:
                    investigate(qno,docs,docids[qno])
        else:
            print("Showing Output Rankings of K-Means :-")
            queries = query_kind["kmeans_ap"]
            good = queries[0]
            bad = queries[1]

            if (typedoc=="good"):
                print("Showing Good Queries:- ")
                for qno in good:
                    investigate(qno,docs,docids[qno])
            else:
                print("Showing Bad Queries:- ")
                for qno in bad:
                    investigate(qno,docs,docids[qno])
        
    else:
        docs = docs_zf
        if (cluster == "clarans"):
            print("Showing Output Rankings of CLARANS :-")
            queries = query_kind["clarans_zf"]
            good = queries[0]
            bad = queries[1]
            
            if (typedoc=="good"):
                print("Showing Good Queries:- ")
                for qno in good:
                    investigate(qno,docs,docids[qno])
            else:
                print("Showing Bad Queries:- ")
                for qno in bad:
                    investigate(qno,docs,docids[qno])
        else:
            print("Showing Output Rankings of K-Means :-")
            queries = query_kind["kmeans_zf"]
            good = queries[0]
            bad = queries[1]

            if (typedoc=="good"):
                print("Showing Good Queries:- ")
                for qno in good:
                    investigate(qno,docs,docids[qno])
            else:
                print("Showing Bad Queries:- ")
                for qno in bad:
                    investigate(qno,docs,docids[qno])

In [492]:
show_queries("ap","good","kmeans")

Showing Output Rankings of K-Means :-
Showing Good Queries:- 
Chosen Query: Topic:  Iran-Contra Affair
First Retrieved Document:
   A grand jury investigating the Iran-Contra
affair on Wednesday considered a prosecutor's request for an
indictment charging criminal violations by key participants in the
worst scandal of the Reagan administration.
   The special federal grand jury that has been investigating the
case nearly 14 months began deliberations after independent counsel
Lawrence E. Walsh made a personal appearance with several of his
top assistants.
   Charges long have been expected against former national security
adviser John M. Poindexter, his one-time aide Lt. Col. Oliver L.
North and two arms dealers, retired Air Force Maj. Gen. Richard V.
Secord and his business partner, Albert Hakim.
   North, who was fired from his job as a National Security Council
aide in late 1986, meanwhile, went to confer with his defense
attorney, Brendan V. Sullivan Jr.
   After making his appeara

In [493]:
show_queries("ap","bad","kmeans")

Showing Output Rankings of K-Means :-
Showing Bad Queries:- 
Chosen Query: Topic:  Poaching
First Retrieved Document:
   Independent Counsel Lawrence E. Walsh is
leaving the door open for further criminal charges in the
Iran-Contra affair following the indictment of Lt. Col. Oliver
North and three others on charges of ``deceitfully exploiting''
U.S. weapons sales to Iran.
   The 23-count indictment charging that North, former national
security adviser John Poindexter and two weapons dealers engaged in
a criminal conspiracy ``is simply an interim report'' of the grand
jury's 14-month investigation, Walsh told reporters.
   ``The grand jury is not finished,'' Walsh said Wednesday after
the panel returned the 101-page indictment against North,
Poindexter and arms dealers Albert Hakim and retired Air Force Maj.
Gen. Richard V. Secord.
   Grand jurors will continue to meet because ``they have
uncompleted investigations,'' Walsh said, refusing to say if more
criminal charges would be brought

In [494]:
show_queries("zf","good","kmeans")

Showing Output Rankings of K-Means :-
Showing Good Queries:- 
Chosen Query: Topic: Fiber Optics Equipment Manufacturers
First Retrieved Document:

Justice Accuses JTN Protesters of Scare Tactics&M

Atotrneys for the Justice Department and AT&T Co. argued recently that the
protest against the department's cancellation of the Justice
Telecommunications Network was designed to scare agencies into sticking with
procurements already in progress rather than switching to the FTS 2000
network.&M

But lawyers for the protesting Computer Sciences Corp. (CSC) told the General
Services Administration Board of Contract Appeals all they wanted for Justice
to meet its obligations to intending bidders.&M

CSC is "not seeking to establish any precedents here," only accountability,
CSC attorney Sol Glasner said.&P  "We are not seeking any new standards here,
we're only seeking the enforcement of the existing federal procurement
standards."&M

"We think this protest is about sending a message to all othe

In [495]:
show_queries("zf","bad","kmeans")

Showing Output Rankings of K-Means :-
Showing Bad Queries:- 
Chosen Query: Topic:  Conflicting Policy 
First Retrieved Document:

NEW FOR MACINTOSH -- Dayna Ships Etherprint SALT LAKE CITY, UTAH, U.S.A.,&O
1990 JUN 19 (NB) -- Dayna Communications Inc. has announced shipment of
EtherPrint, an EtherTalk to LocalTalk protocol converter for LocalTalk
network printers.&M

EtherPrint, with a suggested retail price of $499, is a 5.5-inch x 7.5-inch x
1.5-inch box with a built-in Ethernet transceiver and a LocalTalk port.&P  A
built-in AUI port allows the attachment, through an external transceiver, of
thick Ethernet, twisted-pair or fiber-optic cables to be attached through an
external transceiver.&P  The device contains a power supply, an Ethernet T-
connector, and cables to connect to LocalTalk printers with 8-pin or 9-pin
ports.&P  Macintosh users of the system plug EtherPrint into the printer,
connect the Ethernet cable and then can access the printer directly or
through network print que

In [496]:
show_queries("ap","good","clarans")

Showing Output Rankings of CLARANS :-
Showing Good Queries:- 
Chosen Query: Topic:  Iran-Contra Affair
First Retrieved Document:
   A federal judge Monday granted a government
request to narrow a conspiracy charge in the Iran-Contra indictment
against former National Security Adviser John Poindexter.
   U.S. District Court Judge Harold Greene agreed to drop
allegations of a conspiracy by Poindexter and others to illegally
divert funds to the Contras from the Reagan administration's secret
arms sales to Iran.
   Remaining in the indictment are allegations that Poindexter
conspired to obstruct Congress, make false statements and destroy
official documents in the Iran-Contra scandal. His alleged
co-conspirators were former White House aide Oliver North and
businessmen Richard Secord and Albert Hakim. North enlisted Secord's
help in running arms to the Contras during a congressional ban in
military aid to the Nicaraguan rebel force.
   Poindexter also is accused of two counts of obstructio

In [497]:
show_queries("ap","bad","clarans")

Showing Output Rankings of CLARANS :-
Showing Bad Queries:- 
Chosen Query: Topic:  Airbus Subsidies 
First Retrieved Document:
   A federal judge Monday granted a government
request to narrow a conspiracy charge in the Iran-Contra indictment
against former National Security Adviser John Poindexter.
   U.S. District Court Judge Harold Greene agreed to drop
allegations of a conspiracy by Poindexter and others to illegally
divert funds to the Contras from the Reagan administration's secret
arms sales to Iran.
   Remaining in the indictment are allegations that Poindexter
conspired to obstruct Congress, make false statements and destroy
official documents in the Iran-Contra scandal. His alleged
co-conspirators were former White House aide Oliver North and
businessmen Richard Secord and Albert Hakim. North enlisted Secord's
help in running arms to the Contras during a congressional ban in
military aid to the Nicaraguan rebel force.
   Poindexter also is accused of two counts of obstruction 

In [498]:
show_queries("zf","good","clarans")

Showing Output Rankings of CLARANS :-
Showing Good Queries:- 
Chosen Query: Topic:  Automation
First Retrieved Document:
Honeywell Inc's Industrial Controls Division used automation to
achieve a 70 percent parts reduction, 40 percent product cost
reduction, vastly improved quality, and significantly reduced
process and delivery lead times for the firm's ST 3000 series of
industrial process sensors and transmitters.&P  The new production
line uses robots and a modular, flexible conveyor system to handle
most manufacturing operations without moment-to-moment control by
factory floor personnel.&P  Development and implementation of the
line was accomplished by a multi-discipline team approach that
included top management, design and process engineering personnel,
information systems specialists, skilled works and manufacturing
operations personnel.&P  The system design involved redesign of the
ST 3000 series for automated manufacturing in conjunction with
design of the production system.&M

In [499]:
show_queries("zf","bad","clarans")

Showing Output Rankings of CLARANS :-
Showing Bad Queries:- 
Chosen Query: Topic:  Greenpeace
First Retrieved Document:

NEWS TRACK RUSSIANS TO SELL SWEDES PCs...The Soviet Union is getting in on
the personal computer market-go-round with plans to sell IBM PC AT and XT
compatibles to Sweden next month.&P  A Soviet plant will manufacture about
10,000 PCs this year, about half of which will be exported to Sweden.&M

DIM YOUR RAYS, PLEASE...Things can get pretty unneighborly and confusing in
outer space what with radiation from one rival's satellites interfering with
readings from another rival's satellites.&P  U.S.&P  space scientists think it's
a simple enough request to ask Soviet researchers to clean up their nuclear
reactors' act: the radiation is promoting cosmic pollution.&M

IT'S ALL IN THE CARDS...Thanks to microchip technology and human ingenuity, a
patient's medical history can be recorded on transportable, credit-card sized
plates.&P  The information can be easily read or retr